In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
import os

In [3]:
IMAGE_SIZE = (224,224)
NUM_CLASSES = 2

In [4]:
DATA_DIR = r'C:\Users\geeth\Downloads\PAI700\Data'
DEFECTIVE_DIR = os.path.join(DATA_DIR, 'Unhealthy')
QUALITY_DIR = os.path.join(DATA_DIR, 'Healthy')

In [5]:
# Create ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [6]:
def load_data(directory):
    images = []
    labels = []
    label = 0 if directory == QUALITY_DIR else 1  # Assign label based on directory
    for filename in os.listdir(directory):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMAGE_SIZE)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

In [7]:
# Load data
healthy_images, healthy_labels = load_data(QUALITY_DIR)
defective_images, defective_labels = load_data(DEFECTIVE_DIR)

In [8]:
# Concatenate images and labels
wood_images = np.concatenate([defective_images, healthy_images], axis=0)
wood_labels = np.concatenate([defective_labels, healthy_labels], axis=0)

In [9]:
# Split data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    wood_images, wood_labels, test_size=0.2, random_state=42)

In [10]:

# Normalize pixel values
train_images_normalized = train_images / 255.0
test_images_normalized = test_images / 255.0

In [11]:
# Load MobileNet pre-trained model
base_model = tf.keras.applications.MobileNet(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

In [12]:
# Freeze the base model
base_model.trainable = False

In [13]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')
])

In [14]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
model.fit(train_images_normalized, train_labels,
          batch_size=32,
          epochs=10,
          validation_data=(test_images_normalized, test_labels))

Epoch 1/10




: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_images_normalized, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

30/30 [==============================] - 4s 131ms/step - loss: 0.0146 - accuracy: 0.9957
Test Loss: 0.01455731876194477
Test Accuracy: 0.9957447052001953


In [ ]:
predictions = model.predict(test_images_normalized)
predicted_classes = (predictions > 0.5).astype("int32")

30/30 [==============================] - 4s 123ms/step


In [ ]:
precision = precision_score(test_labels, predicted_classes)
recall = recall_score(test_labels, predicted_classes)
f1 = f1_score(test_labels, predicted_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [ ]:
VAL_DIR = r'C:\Users\geeth\Downloads\PAI700\test_pics\test_pics'
val_images, val_labels = load_data(VAL_DIR)
val_images_normalized = val_images / 255.0


In [ ]:
val_images

array([[[[126., 106., 115.],
         [112.,  95., 105.],
         [111.,  94., 102.],
         ...,
         [ 80.,  59.,  54.],
         [ 71.,  53.,  41.],
         [ 86.,  59.,  50.]],

        [[127., 113., 130.],
         [127., 108., 130.],
         [109.,  95., 112.],
         ...,
         [106.,  85.,  82.],
         [116., 101., 124.],
         [112.,  91.,  70.]],

        [[159., 142., 161.],
         [131., 117., 132.],
         [128., 110., 124.],
         ...,
         [136., 112., 112.],
         [114.,  82.,  87.],
         [114.,  90.,  80.]],

        ...,

        [[175., 141., 155.],
         [231., 200., 208.],
         [194., 158., 168.],
         ...,
         [217., 169., 169.],
         [217., 181., 181.],
         [224., 183., 189.]],

        [[207., 174., 165.],
         [201., 165., 175.],
         [212., 169., 197.],
         ...,
         [214., 173., 179.],
         [225., 184., 190.],
         [232., 192., 201.]],

        [[177., 140., 148.],
       

In [ ]:
predictions_val = model.predict(val_images_normalized)
print(predictions_val)

3/3 [==============================] - 0s 104ms/step
[[2.4027541e-02 9.6770418e-01]
 [8.8726362e-04 9.9846154e-01]
 [2.3314606e-02 9.6337438e-01]
 [2.8850192e-01 4.0719554e-01]
 [6.1425795e-03 9.8851073e-01]
 [3.8387345e-03 9.4002402e-01]
 [1.0104042e-02 9.7092932e-01]
 [1.9704346e-02 9.4839352e-01]
 [4.3462141e-04 9.9837917e-01]
 [1.4890790e-05 9.9927646e-01]
 [3.1133574e-01 6.3899770e-02]
 [1.1809024e-02 9.9409550e-01]
 [4.4437891e-04 9.9980772e-01]
 [7.1750040e-04 9.9079204e-01]
 [3.4413533e-03 9.9361700e-01]
 [1.5557114e-06 9.9999738e-01]
 [2.6493580e-03 9.9421465e-01]
 [5.6074158e-04 9.9929118e-01]
 [1.8045319e-02 9.4518012e-01]
 [5.8836560e-03 9.7691643e-01]
 [1.3833558e-03 9.9656665e-01]
 [4.0213376e-02 9.6279281e-01]
 [6.3194461e-02 9.2401141e-01]
 [2.3480745e-02 9.6840411e-01]
 [8.3110161e-04 9.9192268e-01]
 [8.1167668e-02 8.4422600e-01]
 [5.7632523e-04 9.9891263e-01]
 [1.0185008e-04 9.9914986e-01]
 [5.6802573e-06 9.9997872e-01]
 [4.4201320e-04 9.9838346e-01]
 [2.7151810e-04 9

In [ ]:
predicted_classes = (predictions_val > 0.5).astype("int32")

In [ ]:
predicted_classes

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [ ]:
model.save('mobilenet_v2.h5')

c:\Users\geeth\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the TensorFlow Lite model to a file
tflite_model_path = 'mobilenet_v2.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\geeth\AppData\Local\Temp\tmp2191bov2\assets


INFO:tensorflow:Assets written to: C:\Users\geeth\AppData\Local\Temp\tmp2191bov2\assets
